In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Загрузка данных
### Датасет triplets_data

In [2]:
triplets_data = pd.read_csv('triplets_data.csv', index_col=0)
triplets_data.head()

,user_id,song_id,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1


In [3]:
triplets_data.shape

(48373585, 3)

### Датасет song_data

In [4]:
song_data = pd.read_csv('song_data.csv')
song_data.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [5]:
song_data.shape

(1000000, 5)

#### Удаляем дубли
В данном датасете присутствуют дубли песен, которые необходимо удалить.

In [6]:
song_data.groupby('song_id', as_index=False).agg({'artist_name': 'count'}).rename(columns={'artist_name': 'count_song_id'}) \
                                            .sort_values(by='count_song_id', ascending=False)

,song_id,count_song_id
824667,SOUYQYY12AF72A000F,3
476252,SOLNCTS12A8C1405EF,3
665418,SOQNMCD12A8C1383D4,3
68469,SOBPAEP12A58A77F49,3
827438,SOVASUH12A67021B44,3
...,...,...
333168,SOHXMHZ12A5891FFDF,1
333169,SOHXMIP12AAF3B3C24,1
333170,SOHXMJG12AB0182DA9,1
333171,SOHXMJO12AB018F837,1


In [7]:
song_data = song_data.drop_duplicates(['song_id'])
song_data.shape

(999056, 5)

### Датасет sid_mismatches
#### Удаление неверно объединенных пар song_id и track_id

In [8]:
sid_mismatches = pd.read_csv('sid_mismatches.csv', index_col=0)
sid_mismatches.head()

,song_id,track_id
0,SOCMRBE12AB018C546,TRMMREB12903CEB1B1
1,SOLPHZY12AC468ABA8,TRMMBOC12903CEB46E
2,SONGHTM12A8C1374EF,TRMMITP128F425D8D0
3,SONGXCA12A8C13E82E,TRMMAYZ128F429ECE6
4,SOMBCRC12A67ADA435,TRMMNVU128EF343EED


In [9]:
sid_mismatches.shape

(19093, 2)

In [10]:
song_data = song_data[~song_data.song_id.isin(sid_mismatches.song_id)]
song_data.shape

(980144, 5)

### Датасет unique_tracks

In [11]:
unique_tracks = pd.read_csv('unique_tracks.csv', index_col=0)
unique_tracks.head()

,track_id,song_id,artist_name,song_title
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C,Yerba Brava,Si Vos Querés
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens


In [12]:
unique_tracks.shape

(1000000, 4)

In [13]:
print(f'Количество уникальных song_id: {unique_tracks.song_id.nunique()}')
print(f'Количество уникальных track_id: {unique_tracks.track_id.nunique()}')

Количество уникальных song_id: 999056
Количество уникальных track_id: 1000000


In [14]:
unique_tracks = unique_tracks.drop_duplicates(['song_id'])
unique_tracks.shape

(999056, 4)

### Датасеты msd_beatunes_map, msd-MAGD-genreAssignment
#### Добавляем жанры

In [15]:
song_seed_genre = pd.read_csv('msd_beatunes_map.csv', index_col=0)
song_seed_genre_1 = pd.read_csv('msd_lastfm_map.csv', index_col=0)
song_genre = pd.read_csv('msd-MAGD-genreAssignment.csv', index_col=0)

In [16]:
song_genre.head()

,track_id,genre
0,TRAAAAK128F9318786,Pop_Rock
1,TRAAAAV128F421A322,Pop_Rock
2,TRAAAAW128F429D538,Rap
3,TRAAABD128F429CF47,Pop_Rock
4,TRAAACV128F423E09E,Pop_Rock


In [17]:
song_genre.shape

(422714, 2)

In [18]:
print(f'Количество уникальных жанров {song_genre.genre.nunique()}')

Количество уникальных жанров 21


In [19]:
song_seed_genre.head()

,track_id,seed_genre
0,TRAAAAK128F9318786,Rock
1,TRAAAAV128F421A322,Rock
2,TRAAAAW128F429D538,Hip-Hop
3,TRAAAAY128F42A73F0,World
4,TRAAABD128F429CF47,Rock


In [20]:
song_seed_genre.shape

(677038, 2)

In [21]:
print(f'Количество уникальных поджанров {song_seed_genre.seed_genre.nunique()}')

Количество уникальных поджанров 6151


In [22]:
song_seed_genre_1.head()

,track_id,seed_genre
0,TRAAAAK128F9318786,rock
1,TRAAAAW128F429D538,bayarea
2,TRAAABD128F429CF47,oldies
3,TRAAADJ128F4287B47,rock
4,TRAAADZ128F9348C2E,latin


In [23]:
song_seed_genre_1.shape

(505216, 2)

Объединими датасеты по жанрам в один.

In [24]:
tracks_genre = pd.merge(song_seed_genre, song_genre, on='track_id', how='outer')
tracks_genre = pd.merge(tracks_genre, song_seed_genre_1, on='track_id', how='outer')
tracks_genre.head()

,track_id,seed_genre_x,genre,seed_genre_y
0,TRAAAAK128F9318786,Rock,Pop_Rock,rock
1,TRAAAAV128F421A322,Rock,Pop_Rock,NaN
2,TRAAAAW128F429D538,Hip-Hop,Rap,bayarea
3,TRAAAAY128F42A73F0,World,NaN,NaN
4,TRAAABD128F429CF47,Rock,Pop_Rock,oldies


In [25]:
tracks_genre.shape

(826468, 4)

In [26]:
tracks_genre.loc[(tracks_genre.seed_genre_x.isnull()), 'seed_genre_x'] = tracks_genre['genre']
tracks_genre.loc[(tracks_genre.seed_genre_x.isnull()), 'seed_genre_x'] = tracks_genre['seed_genre_y']
tracks_genre.head()

,track_id,seed_genre_x,genre,seed_genre_y
0,TRAAAAK128F9318786,Rock,Pop_Rock,rock
1,TRAAAAV128F421A322,Rock,Pop_Rock,NaN
2,TRAAAAW128F429D538,Hip-Hop,Rap,bayarea
3,TRAAAAY128F42A73F0,World,NaN,NaN
4,TRAAABD128F429CF47,Rock,Pop_Rock,oldies


In [27]:
tracks_genre = tracks_genre.drop(['genre', 'seed_genre_y'], axis=1).rename(columns={'seed_genre_x': 'genre'})

tracks_genre.head()

,track_id,genre
0,TRAAAAK128F9318786,Rock
1,TRAAAAV128F421A322,Rock
2,TRAAAAW128F429D538,Hip-Hop
3,TRAAAAY128F42A73F0,World
4,TRAAABD128F429CF47,Rock


Добавим к уникальным трекам их жанры.

In [28]:
unique_tracks_gnr = pd.merge(unique_tracks, tracks_genre, on='track_id', how='outer')
unique_tracks_gnr.head()

,track_id,song_id,artist_name,song_title,genre
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night,christmas
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan,Pop
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever,Electronica/Dance
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C,Yerba Brava,Si Vos Querés,International
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens,NaN


In [29]:
unique_tracks_gnr.shape

(999984, 5)

### Объединение song_data и unique_tracks

In [30]:
song_df = pd.merge(song_data, unique_tracks_gnr.drop(['artist_name', 'song_title'], axis=1), on='song_id', how='inner')

In [31]:
song_df.head()

,song_id,title,release,artist_name,year,track_id,genre
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,TRMMMYQ128F932D901,christmas
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,TRMMMKD128F425225D,Pop
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,TRMMMRX128F93187D9,Electronica/Dance
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003,TRMMMCH128F425532C,International
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0,TRMMMWA128F426B589,NaN


In [32]:
song_df.shape

(980144, 7)

### Датасет с характеристиками треков data10000

In [33]:
data10000 = pd.read_csv('data10000.csv', sep=';')
data10000.head()

,artist_name,artist_familarity,artist_hotttnesss,artist_id,artist_location,danceability,duration,energy,loudness,mode,mode_confidence,song_hotttnesss,song_id,tempo,time_signature,time_signature_confidence,title,track_id,year
0,Casual,0.581794,0.401998,ARD7TVE1187B99BFB1,California - LA,0.0,218.93179,0.0,-11.197,0,0.636,0.602120,SOMZWCG12A8C13C480,92.198,4,0.778,I Didn't Mean To,TRAAAAW128F429D538,0
1,The Box Tops,0.630630,0.417500,ARMJAGH1187FB546F3,"Memphis, TN",0.0,148.03546,0.0,-9.843,0,0.430,NaN,SOCIWDW12A8C13D406,121.274,4,0.384,Soul Deep,TRAAABD128F429CF47,1969
2,Sonora Santanera,0.487357,0.343428,ARKRRTF1187B9984DA,NaN,0.0,177.47546,0.0,-9.689,1,0.565,NaN,SOXVLOJ12AB0189215,100.070,1,0.000,Amor De Cabaret,TRAAADZ128F9348C2E,0
3,Adam Ant,0.630382,0.454231,AR7G5I41187FB4CE6C,"London, England",0.0,233.40363,0.0,-9.013,1,0.749,NaN,SONHOTT12A8C13493C,119.293,4,0.000,Something Girls,TRAAAEF128F4273421,1982
4,Gob,0.651046,0.401724,ARXR32B1187FB57099,NaN,0.0,209.60608,0.0,-4.501,1,0.371,0.604501,SOFSOCN12A8C143F5D,129.738,4,0.562,Face the Ashes,TRAAAFD128F92F423A,2007


In [34]:
data10000.shape

(10000, 19)

In [35]:
data10000 = data10000[~data10000.song_id.isin(sid_mismatches.song_id) & ~data10000.track_id.isin(sid_mismatches.track_id)]
data10000.shape

(9766, 19)

# Предобработка данных
## Заполнение пропусков
### Датасет song_df

Для пустых значений жанров заполним пропуски значениями 'No genre'.

In [36]:
song_df.genre = song_df['genre'].fillna('no_genre')
song_df.head()

,song_id,title,release,artist_name,year,track_id,genre
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,TRMMMYQ128F932D901,christmas
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,TRMMMKD128F425225D,Pop
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,TRMMMRX128F93187D9,Electronica/Dance
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003,TRMMMCH128F425532C,International
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0,TRMMMWA128F426B589,no_genre


In [37]:
song_df[(song_df.genre == 'no_genre')].shape

(170647, 7)

170647 трека не имеют жанра.
  
Переведем все названия жанров в нижний регистор, так как в результате объединения датасетов получились жанры 'Pop' и 'pop', являющиеся одним жанром.

In [38]:
song_df['genre'] = song_df['genre'].str.lower().replace('-', '', regex=True) 
song_df.head()

,song_id,title,release,artist_name,year,track_id,genre
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,TRMMMYQ128F932D901,christmas
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,TRMMMKD128F425225D,pop
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,TRMMMRX128F93187D9,electronica/dance
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003,TRMMMCH128F425532C,international
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0,TRMMMWA128F426B589,no_genre


Заполним пропуски года для треков из одного альбома известными значениями.

In [39]:
print(f'Количество треков, для которых определен год: {song_df.song_id[song_df.year > 0].count()}')

Количество треков, для которых определен год: 503890


In [40]:
song_release_year = song_df[song_df.year != 0][['release', 'artist_name', 'year']].drop_duplicates() \
                                                                                  .rename(columns={'year': 'year_'})
song_release_year.head()

,release,artist_name,year_
0,Monster Ballads X-Mas,Faster Pussy cat,2003
1,Karkuteillä,Karkkiautomaatti,1995
2,Butter,Hudson Mohawke,2006
3,De Culo,Yerba Brava,2003
7,Da Bomb,Kris Kross,1993


У некоторых релизов несколько годов выпуска. Поэтому заполним пропуски первым годом выпуска релиза.

In [41]:
song_release_year.groupby(['release', 'artist_name'], as_index=False).agg({'year_': 'count'}) \
                                                                     .sort_values('year_', ascending=False).head()

,release,artist_name,year_
96526,The Complete Stevie Wonder,Stevie Wonder,18
20317,Clouds In My Coffee 1965-1995,Carly Simon,16
51129,Kaikkien aikojen parhaat -45 klassikkoa - Aatt...,Jope Ruonansuu,14
8055,Anthology,Charley Pride,13
116353,Xxv,Vader,12


In [42]:
song_release_year = song_release_year.sort_values('year_', ascending=False) \
                                     .groupby(['release', 'artist_name'], as_index=False).first()
song_release_year.head()

,release,artist_name,year_
0,Em Are I,Jeffrey Lewis & The Junkyard,2009
1,"fifties"" Million Sellers",The Weavers,1988
2,A Tribute Album,Daron Norwood,1994
3,A Tribute Album,Diamond Rio,1994
4,A Tribute Album,Joe Diffie,1994


In [43]:
song_df = pd.merge(song_df, song_release_year, on=['release', 'artist_name'], how='left').fillna(0)
song_df.head()

,song_id,title,release,artist_name,year,track_id,genre,year_
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,TRMMMYQ128F932D901,christmas,2003.0
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,TRMMMKD128F425225D,pop,1995.0
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,TRMMMRX128F93187D9,electronica/dance,2009.0
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003,TRMMMCH128F425532C,international,2003.0
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0,TRMMMWA128F426B589,no_genre,0.0


In [44]:
song_df.loc[(song_df.year == 0), 'year'] = song_df['year_']
song_df.head()

,song_id,title,release,artist_name,year,track_id,genre,year_
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003,TRMMMYQ128F932D901,christmas,2003.0
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995,TRMMMKD128F425225D,pop,1995.0
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006,TRMMMRX128F93187D9,electronica/dance,2009.0
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003,TRMMMCH128F425532C,international,2003.0
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0,TRMMMWA128F426B589,no_genre,0.0


In [45]:
song_df = song_df.drop('year_', axis=1)
song_df[song_df.year == 0].shape

(372225, 7)

Однако, для 372225 треков заполнить год таким образом не удалось.

In [46]:
# song_df.to_csv('song_df.csv')

## Объединение всех датасетов

In [47]:
user_song_df = pd.merge(triplets_data, song_df, on='song_id', how='inner')
user_song_df.head()

,user_id,song_id,play_count,title,release,artist_name,year,track_id,genre
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,Nothing from Nothing,To Die For,Billy Preston,1974,TRIRLYL128F42539D1,r&b
1,fd13b9d49c54e00ff413fe3c095ba581c7fc611e,SOAPDEY12A81C210A9,8,Nothing from Nothing,To Die For,Billy Preston,1974,TRIRLYL128F42539D1,r&b
2,35752a9e4d917c8fc5da88e90c25adc6b52220b6,SOAPDEY12A81C210A9,1,Nothing from Nothing,To Die For,Billy Preston,1974,TRIRLYL128F42539D1,r&b
3,bfe8d6d1e32d9de12c632d68712e7e4a23416e93,SOAPDEY12A81C210A9,1,Nothing from Nothing,To Die For,Billy Preston,1974,TRIRLYL128F42539D1,r&b
4,46d3ef1b4f317aba8f0226e937c67ecef9fa2e42,SOAPDEY12A81C210A9,4,Nothing from Nothing,To Die For,Billy Preston,1974,TRIRLYL128F42539D1,r&b


In [48]:
user_song_df.shape

(45795099, 9)

In [49]:
user_song_df.to_csv('user_song_df.csv')